In [1]:
from PIL import Image
import torch
import argparse
import numpy as np
import matplotlib.pyplot as plt
import sys
from torchsummary import summary
import cv2



In [12]:
repo_path = '/workspace/stereo-transformer'
pretrained_weight_path = '../../PretrainedWeights/kitti_finetuned_model.pth.tar'
sys.path.append(repo_path) # add relative path

In [13]:
import ipyvolume as ipv
from module.sttr import STTR
from dataset.preprocess import normalization, compute_left_occ_region
from utilities.misc import NestedTensor

def visualize_3D_points_in_jupyter(points_3D, size=2, marker="sphere"):
    # Assuming points_3D is a N x 3 numpy array
    x = points_3D[:, 0]
    y = points_3D[:, 1]
    z = points_3D[:, 2]

    ipv.quickscatter(x, y, z, size=size, marker=marker)
    ipv.show()

from module.sttr import STTR
from dataset.preprocess import normalization, compute_left_occ_region
from utilities.misc import NestedTensor

def load_STTR_model(repo_path, pretrained_weight_path):
    # Default parameters
    args = type('', (), {})() # create empty args
    args.channel_dim = 128
    args.position_encoding='sine1d_rel'
    args.num_attn_layers=6
    args.nheads=8
    args.regression_head='ot'
    args.context_adjustment_layer='cal'
    args.cal_num_blocks=8
    args.cal_feat_dim=16
    args.cal_expansion_ratio=4


    model = STTR(args).cuda().eval()

    # Load the pretrained model
    model_file_name = pretrained_weight_path
    checkpoint = torch.load(model_file_name)
    pretrained_dict = checkpoint['state_dict']
    model.load_state_dict(pretrained_dict, strict=False) # prevent BN parameters from breaking the model loading
    print("Pre-trained model successfully loaded.")
    
    return model

def STTR_input_factory(left_img, right_img):
    # donwsample attention by stride of 3
    left = left_img
    right = right_img
    h, w, _ = left.shape
    bs = 1
    
    downsample = 3
    col_offset = int(downsample / 2)
    row_offset = int(downsample / 2)
    sampled_cols = torch.arange(col_offset, w, downsample)[None,].expand(bs, -1).cuda()
    sampled_rows = torch.arange(row_offset, h, downsample)[None,].expand(bs, -1).cuda()

    input_data = {'left': left, 'right':right}
    input_data = normalization(**input_data)

    # build NestedTensor
    input_data = NestedTensor(input_data['left'].cuda()[None,],input_data['right'].cuda()[None,], sampled_cols=sampled_cols, sampled_rows=sampled_rows)
    return input_data


import cv2

def disparity_to_3D_points(left_prj, right_prj, disparity):
    """
    Convert disparity map to 3D points using the given projection matrices.

    Parameters:
    - left_prj: 3x4 numpy array representing the left rectified projection matrix.
    - right_prj: 3x4 numpy array representing the right rectified projection matrix.
    - disparity: 2D numpy array representing the disparity map.

    Returns:
    - 3D numpy array with shape (H, W, 3) representing the 3D points.
    """
    # Compute Q matrix from projection matrices
    f = left_prj[0, 0]  # Focal length, assuming it's the same for both cameras after rectification
    T = right_prj[0, 3] / f  # Baseline

    Q = np.array([
        [1, 0, 0, -0.5 * disparity.shape[1]],
        [0, -1, 0, 0.5 * disparity.shape[0]],
        [0, 0, 0, -f],
        [0, 0, -1 / T, 0]
    ])

    # Reproject image to 3D
    points_3D = cv2.reprojectImageTo3D(disparity, Q)
    return points_3D

In [14]:
model = load_STTR_model(repo_path, pretrained_weight_path)

Pre-trained model successfully loaded.


In [5]:
import os
def load_images_from_directory(directory_path):
    # Get all files from the directory
    all_files = os.listdir(directory_path)
    
    # Filter out non-image files if necessary (optional)
    image_files = [f for f in all_files if f.endswith('.png') or f.endswith('.jpg') or f.endswith('.jpeg')]
    
    # Read images and append to a list
    images = [cv2.imread(os.path.join(directory_path, image_file)) for image_file in image_files]
    
    # Convert list of images to numpy array
    return np.array(images)

def load_depth_data_from_directory(directory_path):
    # Get all .npy files from the directory
    npy_files = [f for f in os.listdir(directory_path) if f.endswith('.npy')]
    
    # Load each .npy file and append to a list
    depth_data_list = [np.load(os.path.join(directory_path, npy_file)) for npy_file in npy_files]
    
    return np.array(depth_data_list)

class TartanData:
    def __init__(self, repo_path, scene_name, part_id, difficulty='Easy'):
        self.fx = 320.0  # focal length x
        self.fy = 320.0  # focal length y
        self.cx = 320.0  # optical center x
        self.cy = 240.0  # optical center y

        self.fov = 90 #deg # field of view

        self.width = 640
        self.height = 480
        
        path = repo_path + '/' + scene_name + '/' 
        path += scene_name +'/' + difficulty + '/' + "P" + str(part_id).zfill(3) + '/'
        self.left_images = load_images_from_directory(path + 'image_left')
        self.right_images = load_images_from_directory(path + 'image_right')
        self.left_depth = load_depth_data_from_directory(path + 'depth_left')
        self.right_depth = load_depth_data_from_directory(path + 'depth_right')
        

In [6]:
tartan_data = TartanData("/workspace/TartanAir", "carwelding", 1)

FileNotFoundError: [Errno 2] No such file or directory: '/workspace/TartanAir/carwelding/carwelding/Easy/P001/image_left'

In [7]:
left_images = tartan_data.left_images
right_images = tartan_data.right_images
left_depths_GT = tartan_data.left_depth
right_depths_GT = tartan_data.right_depth


print(right_images.shape)
print(left_images.shape)
print(left_depths_GT.shape)
print(right_depths_GT.shape)

normalize = lambda img: (img - img.min()) / (img.max() - img.min())

idx = 200

fig, axes = plt.subplots(2, 2, figsize=(20, 20))
axes[0, 0].imshow(left_images[idx])
axes[0, 1].imshow(right_images[idx])
axes[1, 0].imshow(normalize(left_depths_GT[idx]), cmap='gray')
axes[1, 1].imshow(normalize(right_depths_GT[idx]), cmap='gray')
plt.show()

NameError: name 'tartan_data' is not defined

In [8]:
idx = 200
input_data = STTR_input_factory(left_images[idx], right_images[idx])

with torch.no_grad():
    output = model(input_data)
    
disp_map = output['disp_pred'].data.cpu().numpy()[0]
occ_map = output['occ_pred'].data.cpu().numpy()[0] > 0.5
disp_map[occ_map] = 0.0

# culling
disp_map = disp_map[:350, 100:1100]

# points_3D = disparity_to_3D_points(left_prj, right_prj, disp_map)

# # Mask out points where disparity is 0 (or less) as they're likely invalid
# mask = disp_map > 0
# valid_points_3D = points_3D[mask]

print(valid_points_3D.shape)

plt.figure(1)
plt.imshow(left_images[idx])
plt.figure(2)
plt.imshow(disp_map)
plt.show()

# visualize_3D_points_in_jupyter(valid_points_3D, size=1, marker='sphere')

NameError: name 'left_images' is not defined